In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np

## load the libraries 
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Input, Conv2D, LSTM, MaxPool2D, UpSampling2D, Flatten, Dropout, concatenate,GlobalAveragePooling2D, AveragePooling2D, Activation, Add, LeakyReLU, PReLU, ELU, ReLU, ThresholdedReLU
from keras.layers.normalization import BatchNormalization
from keras.activations import relu
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.models import Model
from keras.initializers import glorot_uniform, Constant
from keras import optimizers
from keras.preprocessing.image import img_to_array, array_to_img
from keras.optimizers import Optimizer
from keras.utils.generic_utils import get_custom_objects
from keras_tqdm import TQDMNotebookCallback

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from hyperopt import Trials, STATUS_OK, tpe

import gc;

from hyperas import optim
from hyperas.distributions import choice, uniform, lognormal, normal, loguniform

from hyperas.distributions import uniform

C:\Users\Infernal\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    nb_classes = 10
    # the data, shuffled and split between train and test sets
    train = pd.read_csv("data/fashion/fashion-mnist_train.csv")
    test = pd.read_csv("data/fashion/fashion-mnist_test.csv")

    train_x = train[list(train.columns)[1:]].values
    train_y = train['label'].values
    test_x = test[list(test.columns)[1:]].values
    test_y = test['label'].values

    train_x=train_x.reshape(60000,28,28,1)
    test_x=test_x.reshape(10000,28,28,1)
    
    train_y = to_categorical(train_y, num_classes = 10)
    test_y = to_categorical(test_y, num_classes = 10)
    
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=int(train_x.shape[0]/5))


    train_x = train_x / 255
    val_x = val_x / 255
    test_x = test_x / 255

    return train_x, train_y, val_x, val_y, test_x, test_y

In [3]:
def main_block(x, filters, n, strides, activation, initializer, dropout):
    # Normal part
    x_res = Conv2D(filters, (3,3), strides=strides, padding="same", kernel_initializer=initializer, kernel_regularizer=l2(5e-4))(x)
    x_res = BatchNormalization()(x_res)
    x_res = activation()(x_res)
    x_res = Conv2D(filters, (3,3), padding="same", kernel_initializer=initializer)(x_res)
    # Alternative branch
    x = Conv2D(filters, (1,1), strides=strides)(x)
    # Merge Branches
    x = Add()([x_res, x])

    for i in range(n-1):
        # Residual conection
        x_res = BatchNormalization()(x)
        x_res = activation()(x_res)
        x_res = Conv2D(filters, (3,3), padding="same", kernel_initializer=initializer)(x_res)
        # Apply dropout if given
        if dropout: x_res = Dropout(rate=dropout)(x)
        # Second part
        x_res = BatchNormalization()(x_res)
        x_res = activation()(x_res)
        x_res = Conv2D(filters, (3,3), padding="same", kernel_initializer=initializer)(x_res)
        # Merge branches
        x = Add()([x, x_res])

    # Inter block part
    x = BatchNormalization()(x)
    x = activation()(x)
    return x

def build_model(input_dims, output_dim, n, k, activation, initializer, dropout=None):
    assert (n-4)%6 == 0
    assert k%2 == 0
    n = (n-4)//6 
    
    #1)Input
    inputs = Input(shape=(input_dims))

    #2)Initial part
    x = Conv2D(16, (3,3), padding="same", kernel_initializer=initializer)(inputs)
    x = BatchNormalization()(x)
    x = activation()(x)

    #3)wide blocks (can be made into for loop)
    x = main_block(x, 16*k, n, (1,1), activation, initializer, dropout)
    x = main_block(x, 32*k, n, (2,2), activation, initializer, dropout)
    x = main_block(x, 64*k, n, (2,2), activation, initializer, dropout)

    #4)Final part
    x = AveragePooling2D((7,7))(x)
    x = Flatten()(x)
    outputs = Dense(output_dim, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [9]:
def model(train_x, train_y, val_x, val_y, test_x, test_y):
    architecture={{choice([(22,8), (28,10), (40,4)])}}
    activation={{choice([ReLU, LeakyReLU, PReLU, ELU, ThresholdedReLU])}}
    initializer={{choice(['he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal'])}}
    dropout={{uniform(0, 0.5)}}
    
    learning_rate=10**{{uniform(-4, -1)}}
    opt={{choice([optimizers.Adam, optimizers.RMSprop, optimizers.Adadelta, optimizers.Nadam, optimizers.Adamax])}}
    if architecture[0]==28:
        batch={{choice([32, 64])}}
    else:
        batch={{choice([32, 64, 128])}}
    
    
    print("""
          Hyperparameters Used \n\n\n
          -----------------------------
          model: ({},{})
          activation: {}
          initializer: {}
          dropout: {}
          learning rate: {}
          optimizer: {}
          batch size: {}
          -----------------------------
          """
          .format(architecture[0], architecture[1], activation, initializer, dropout, learning_rate, opt, batch))
    
    model = build_model((28,28,1), 10, architecture[0], architecture[1], activation, initializer, dropout)
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0),
                 ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=0),
                 ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5,min_lr=0.00001),
                 TQDMNotebookCallback()]

    model.compile(optimizer=opt(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_x, train_y, validation_data = (val_x, val_y), callbacks=callbacks, batch_size=batch, epochs=20, verbose=0)

    score, acc = model.evaluate(test_x, test_y, verbose=0)
    
    y_val_pred = model.predict(val_x)
    print("Validation accuracy: ", accuracy_score(np.argmax(val_y, axis=1), np.argmax(y_val_pred, axis=1)))
    
    y_test_pred = model.predict(test_x)
    print("Test accuracy: ", accuracy_score(np.argmax(test_y, axis=1), np.argmax(y_test_pred, axis=1)), "\n\n")
    
    K.clear_session()
    gc.collect()
    del model
    
    return {'loss': acc, 'status': STATUS_OK}


In [10]:
if __name__ == '__main__':
    functions=[main_block, build_model]

    best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          functions=functions,
                                          algo=tpe.suggest,
                                          max_evals=30,
                                          trials=Trials(),
                                          notebook_name="WRN Hyperas")

    train_x, train_y, val_x, val_y, test_x, test_y = data()
    print("Best run: ", best_run)

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import keras.backend as K
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers import Dense, Input, Conv2D, LSTM, MaxPool2D, UpSampling2D, Flatten, Dropout, concatenate, GlobalAveragePooling2D, AveragePooling2D, Activation, Add, LeakyReLU, PReLU, ELU, ReLU, ThresholdedReLU
except:
    pass

try:
    from keras.layers.normalization import BatchNormalization
except:
    pass

try:
    from keras.activations import relu
except:
    pass

try:
    from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
except:
    pass

try:
    from keras.utils import to_categorical
except:
    pass

try:
    from keras.regularizers import l2
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.initializ



                                                    
Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.005978429224342108.


  0%|          | 0/30 [37:42<?, ?it/s, best loss: ?]



Validation accuracy:                                


  0%|          | 0/30 [39:56<?, ?it/s, best loss: ?]

0.9153333333333333                                  


  0%|          | 0/30 [39:56<?, ?it/s, best loss: ?]

Test accuracy:                                      


  0%|          | 0/30 [40:03<?, ?it/s, best loss: ?]

0.9185                                              


  0%|          | 0/30 [40:03<?, ?it/s, best loss: ?]

  3%|▎         | 1/30 [40:03<19:21:37, 2403.36s/it, best loss: 0.9185]

                                                                      
          Hyperparameters Used 



          -----------------------------
          model: (40,4)
          activation: <class 'keras.layers.advanced_activations.LeakyReLU'>
          initializer: glorot_uniform
          dropout: 0.41833334235579095
          learning rate: 0.05476313990663524
          optimizer: <class 'keras.optimizers.Adamax'>
          batch size: 128
          -----------------------------
 



                                                                      
Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.027381569147109985.


  3%|▎         | 1/30 [1:07:13<19:21:37, 2403.36s/it, best loss: 0.9185]



Validation accuracy:                                                    


  3%|▎         | 1/30 [1:16:27<19:21:37, 2403.36s/it, best loss: 0.9185]

0.9131666666666667                                                      


  3%|▎         | 1/30 [1:16:27<19:21:37, 2403.36s/it, best loss: 0.9185]

Test accuracy:                                                          


  3%|▎         | 1/30 [1:16:34<19:21:37, 2403.36s/it, best loss: 0.9185]

0.9158                                                                  


  3%|▎         | 1/30 [1:16:34<19:21:37, 2403.36s/it, best loss: 0.9185]

  7%|▋         | 2/30 [1:16:34<18:11:53, 2339.78s/it, best loss: 0.9158]

                                                                        
          Hyperparameters Used 



          -----------------------------
          model: (28,10)
          activation: <class 'keras.layers.advanced_activations.ThresholdedReLU'>
          initializer: glorot_uniform
          dropout: 0.23634271098298

...



Validation accuracy:                                                     


 90%|█████████ | 27/30 [33:09:07<4:02:40, 4853.66s/it, best loss: 0.6673]

0.6935                                                                   


 90%|█████████ | 27/30 [33:09:07<4:02:40, 4853.66s/it, best loss: 0.6673]

Test accuracy:                                                           


 90%|█████████ | 27/30 [33:09:28<4:02:40, 4853.66s/it, best loss: 0.6673]

0.7074                                                                   


 90%|█████████ | 27/30 [33:09:28<4:02:40, 4853.66s/it, best loss: 0.6673]

 93%|█████████▎| 28/30 [33:09:28<3:02:58, 5489.12s/it, best loss: 0.6673]

                                                                         
          Hyperparameters Used 



          -----------------------------
          model: (40,4)
          activation: <class 'keras.layers.advanced_activations.ReLU'>
          initializer: he_uniform
          dropout: 0.1498182282337851
   



                                                                         
Epoch 00013: ReduceLROnPlateau reducing learning rate to 5.5283759138546884e-05.


 93%|█████████▎| 28/30 [33:38:01<3:02:58, 5489.12s/it, best loss: 0.6673]



                                                                         
Epoch 00017: ReduceLROnPlateau reducing learning rate to 2.7641879569273442e-05.


 93%|█████████▎| 28/30 [33:46:43<3:02:58, 5489.12s/it, best loss: 0.6673]



Validation accuracy:                                                     


 93%|█████████▎| 28/30 [33:51:17<3:02:58, 5489.12s/it, best loss: 0.6673]

0.9273333333333333                                                       


 93%|█████████▎| 28/30 [33:51:17<3:02:58, 5489.12s/it, best loss: 0.6673]

Test accuracy:                                                           


 93%|█████████▎| 28/30 [33:51:23<3:02:58, 5489.12s/it, best loss: 0.6673]

0.9338                                                                   


 93%|█████████▎| 28/30 [33:51:23<3:02:58, 5489.12s/it, best loss: 0.6673]

 97%|█████████▋| 29/30 [33:51:23<1:16:37, 4597.09s/it, best loss: 0.6673]

                                                                         
          Hyperparameters Used 



          -----------------------------
          model: (40,4)
          activation: <class 'keras.layers.advanced_activations.PReLU'>
          initializer: he_uniform
          dropout: 0.029639849065738244




Validation accuracy:                                                     


 97%|█████████▋| 29/30 [34:50:07<1:16:37, 4597.09s/it, best loss: 0.6673]

0.80525                                                                  


 97%|█████████▋| 29/30 [34:50:07<1:16:37, 4597.09s/it, best loss: 0.6673]

Test accuracy:                                                           


 97%|█████████▋| 29/30 [34:50:15<1:16:37, 4597.09s/it, best loss: 0.6673]

0.8123                                                                   


 97%|█████████▋| 29/30 [34:50:15<1:16:37, 4597.09s/it, best loss: 0.6673]

100%|██████████| 30/30 [34:50:15<00:00, 4277.51s/it, best loss: 0.6673]  Best run:  {'activation': 2, 'architecture': 1, 'batch': 1, 'batch_1': 0, 'dropout': 0.13504869220348778, 'dropout_1': -3.966450318495895, 'initializer': 0, 'opt': 2}
